In [1]:
from utils import alsoknownas, birthdate, deathday, nameofperson
import numpy as np
import pandas as pd
import re, nltk, spacy, gensim
from utils import alsoknownas, birthdate, deathday, nameofperson
# Sklearn
from pprint import pprint
# Plotting tools
import pyLDAvis
import pyLDAvis.gensim
import matplotlib.pyplot as plt
%matplotlib inline
import pickle
#Gensim Libraries
import pandas as pd
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim.parsing.preprocessing import preprocess_string, strip_tags, strip_punctuation, strip_numeric, remove_stopwords
from gensim.corpora.dictionary import Dictionary
from gensim.utils import tokenize
from gensim.models.ldamodel import LdaModel
from gensim.models import HdpModel

In [2]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

In [3]:
df = pd.read_csv('trainingDataset.csv',encoding='utf8')

In [4]:
df

,date,title,category,link,abstract,paragraphs
0,21.04.2020,Barack Obama,NaN,https://en.wikipedia.org/wiki/Barack_Obama,Barack Hussein Obama II ( (listen); born Augus...,['Barack Hussein Obama II ( (listen); born Aug...
1,21.04.2020,Joe Biden,NaN,https://en.wikipedia.org/wiki/Joe_Biden,Joseph Robinette Biden Jr. (; born November 20...,['Joseph Robinette Biden Jr. (; born November ...
2,21.04.2020,George W. Bush,NaN,https://en.wikipedia.org/wiki/George_W._Bush,"George Walker Bush (born July 6, 1946) is an A...","['George Walker Bush (born July 6, 1946) is an..."
3,21.04.2020,Donald Trump,NaN,https://en.wikipedia.org/wiki/Donald_Trump,"Donald John Trump (born June 14, 1946) is the ...","['Donald John Trump (born June 14, 1946) is th..."
4,21.04.2020,George Washington,NaN,https://en.wikipedia.org/wiki/George_Washington,"George Washington (February 22, 1732 – Decembe...","['George Washington (February 22, 1732 – Decem..."
...,...,...,...,...,...,...
1600,21.04.2020,Pablo Squella,NaN,https://en.wikipedia.org/wiki/Pablo_Squella,"Pablo Squella Serrano (born August 14, 1963) i...","['Pablo Squella Serrano (born August 14, 1963)..."
1601,21.04.2020,Ulises de la Cruz,NaN,https://en.wikipedia.org/wiki/Ulises_de_la_Cruz,Ulises Hernán de la Cruz Bernardo (born 8 Febr...,"[""Ulises Hernán de la Cruz Bernardo (born 8 Fe..."
1602,21.04.2020,Cecilia Tait,NaN,https://en.wikipedia.org/wiki/Cecilia_Tait,"Cecilia Roxana Tait Villacorta (born May 2, 19...","['Cecilia Roxana Tait Villacorta (born May 2, ..."
1603,21.04.2020,Cenaida Uribe,NaN,https://en.wikipedia.org/wiki/Cenaida_Uribe,Cenaida Cebastiana Uribe Medina (born December...,['Cenaida Cebastiana Uribe Medina (born Decemb...


In [5]:
# Converting data to list
data = df.abstract.values.tolist()

In [6]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations
data_words = list(sent_to_words(data))
print(data_words[:1])

[['barack', 'hussein', 'obama', 'ii', 'listen', 'born', 'august', 'is', 'an', 'american', 'politician', 'and', 'attorney', 'who', 'served', 'as', 'the', 'th', 'president', 'of', 'the', 'united', 'states', 'from', 'to']]


In [7]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# A trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['barack', 'hussein', 'obama', 'ii', 'listen', 'born', 'august', 'is', 'an', 'american', 'politician', 'and', 'attorney', 'who', 'served', 'as', 'the', 'th', 'president', 'of', 'the', 'united_states', 'from', 'to']


In [8]:
# Define functions for removing stopwords, making bigrams, making trigrams and performing lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [9]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en_core_web_sm' model, keeping only tagger component (for efficiency)
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['listen', 'bear', 'american', 'politician', 'attorney', 'serve']]


In [10]:
# Create Dictionary based on lemmatized data
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View the corpus
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1)]]


In [11]:
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('american', 1),
  ('attorney', 1),
  ('bear', 1),
  ('listen', 1),
  ('politician', 1),
  ('serve', 1)]]

In [24]:
#Fit the corpus to the HDP Model
hdpmodel = HdpModel(corpus=corpus, id2word=id2word)
#View the topics
hdpmodel.show_topics()

C:\Users\Shilpu\Anaconda3\lib\site-packages\gensim\models\hdpmodel.py:467: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  start_time = time.clock()


[(0,
  '0.005*filmmaker + 0.004*ольга + 0.004*gower + 0.004*oskari + 0.004*early + 0.004*travancore + 0.003*military + 0.003*stint + 0.003*academia + 0.003*locate + 0.003*occasion + 0.003*czech + 0.003*indian + 0.003*eye + 0.003*lorraine + 0.003*company + 0.003*proprietor + 0.003*vegetable + 0.003*publisher + 0.003*resign'),
 (1,
  '0.004*romilly + 0.004*similar + 0.004*bull + 0.004*discover + 0.004*recording + 0.004*job + 0.004*glendenne + 0.004*meissen + 0.003*اينوكي + 0.003*division + 0.003*agricultural + 0.003*excavation + 0.003*charter + 0.003*scriptwriter + 0.003*draft + 0.003*leyte + 0.003*sportsperson + 0.003*muthu + 0.003*καρασεβδας + 0.003*iranian'),
 (2,
  '0.004*support + 0.004*group + 0.004*swan + 0.004*radical + 0.004*pinarayi + 0.003*fianna_fail + 0.003*academic + 0.003*aleksandr + 0.003*teacher + 0.003*dismiss + 0.003*friend + 0.003*birth + 0.003*franchise + 0.003*hoffmeister + 0.003*australian_rule + 0.003*spell + 0.003*solicit + 0.003*назарович + 0.003*soap + 0.003*co

In [25]:
hdptopics = hdpmodel.show_topics(formatted=False)

In [26]:
alpha,beta=hdpmodel.hdp_to_lda()
lda_model = LdaModel(id2word=id2word,num_topics=len(alpha), alpha=alpha, eta=hdpmodel.m_eta)
print(lda_model.log_perplexity(corpus))

-167.5853591223431


In [28]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -167.9999049865193

Coherence Score:  0.7774363394059958


In [16]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
137   -0.000387  0.000456       1        1  1.649609
18     0.000017  0.000103       2        1  1.576540
83    -0.000436  0.000030       3        1  1.556443
17     0.000067  0.000257       4        1  1.420342
77     0.000060  0.000082       5        1  1.312795
...         ...       ...     ...      ...       ...
119    0.000320  0.000292     146        1  0.265622
6     -0.000179  0.000337     147        1  0.256618
95    -0.000026 -0.000074     148        1  0.240207
128    0.000083  0.000106     149        1  0.228878
89     0.000228  0.000244     150        1  0.201184

[150 rows x 5 columns], topic_info=             Term      Freq     Total  Category  logprob  loglift
1320  territorial  5.000000  5.000000   Default  30.0000  30.0000
1391  philologist  5.000000  5.000000   Default  29.0000  29.0000
204       founder  5.000000  5.000000   Default  28.0000  28.0000
578       girling  5.000000  5.000000   Default  27.0000  27.0000
371    uppalapati  5.000000  5.000000   Default  26.0000  26.0000
...           ...       ...       ...       ...      ...      ...
1207     surveyor  0.012723  5.139831  Topic150  -7.1630   0.2074
1222     pembroke  0.012649  5.112808  Topic150  -7.1688   0.2068
1238         face  0.012805  5.181933  Topic150  -7.1566   0.2056
382         share  0.012906  5.241940  Topic150  -7.1487   0.2020
7       represent  0.012811  5.213119  Topic150  -7.1561   0.2000

[5006 rows x 6 columns], token_table=Empty DataFrame
Columns: [Topic, Freq, Term]
Index: [], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[138, 19, 84, 18, 78, 77, 27, 83, 105, 60, 59, 9, 64, 13, 82, 46, 25, 119, 47, 94, 5, 50, 15, 34, 74, 32, 28, 73, 69, 8, 10, 22, 4, 63, 91, 37, 97, 1, 36, 48, 11, 111, 57, 92, 24, 72, 109, 103, 31, 142, 132, 133, 17, 107, 118, 106, 39, 137, 101, 38, 95, 76, 40, 134, 66, 42, 30, 41, 53, 14, 139, 128, 3, 71, 61, 75, 143, 116, 35, 44, 23, 93, 54, 45, 123, 140, 104, 51, 150, 43, 29, 99, 115, 98, 148, 81, 68, 108, 121, 16, 20, 56, 80, 88, 130, 114, 117, 110, 87, 12, 65, 102, 141, 79, 89, 145, 49, 100, 131, 126, 55, 67, 85, 70, 136, 62, 124, 144, 6, 147, 58, 86, 127, 2, 146, 122, 113, 33, 26, 52, 21, 112, 135, 125, 149, 120, 7, 96, 129, 90])

In [43]:
#View top 5 words across topics
for idx, topic in lda_model.show_topics(formatted=False, num_words= 5):
    print('Topic: {} \nWords: {}'.format(idx, [w[0] for w in topic]))

Topic: 20 
Words: ['folk', 'belgian', 'basketball', 'raise', 'writer']
Topic: 39 
Words: ['chairman', 'financier', 'viscount', 'voivodship', 'owner']
Topic: 137 
Words: ['ˈmodiː', 'sportsperson', 'act', 'work', 'fitzmaurice']
Topic: 96 
Words: ['rock', 'vayalar', 'low', 'monetary', 'ekanayake']
Topic: 52 
Words: ['scriptwriter', 'colony', 'rise', 'reformer', 'rugby']
Topic: 14 
Words: ['mountaineer', 'ring', 'qualification', 'feel', 'professor']
Topic: 35 
Words: ['locket', 'country', 'donegal', 'савченко', 'rioli']
Topic: 47 
Words: ['follow', 'beddoe', 'language', 'international', 'ram']
Topic: 23 
Words: ['carbohydrate', 'filipina', 'die', 'right', 'face']
Topic: 121 
Words: ['resignation', 'silent', 'billionaire', 'italian', 'owner']


In [40]:
for i,topic in hdpmodel.show_topics(formatted=True, num_topics=len(alpha), num_words=10):
    print(str(i)+": "+ topic)
    print()

0: 0.005*filmmaker + 0.004*ольга + 0.004*gower + 0.004*oskari + 0.004*early + 0.004*travancore + 0.003*military + 0.003*stint + 0.003*academia + 0.003*locate

1: 0.004*romilly + 0.004*similar + 0.004*bull + 0.004*discover + 0.004*recording + 0.004*job + 0.004*glendenne + 0.004*meissen + 0.003*اينوكي + 0.003*division

2: 0.004*support + 0.004*group + 0.004*swan + 0.004*radical + 0.004*pinarayi + 0.003*fianna_fail + 0.003*academic + 0.003*aleksandr + 0.003*teacher + 0.003*dismiss

3: 0.005*mixed + 0.004*norwegian + 0.004*manager + 0.004*town + 0.004*commercial + 0.004*exeter + 0.004*global + 0.003*administration + 0.003*promote + 0.003*title

4: 0.005*gower + 0.005*truck + 0.004*numerous + 0.004*economist + 0.004*lobbyist + 0.004*series + 0.004*rally + 0.004*menzie + 0.003*park + 0.003*notably

5: 0.007*ˈruði + 0.004*bad + 0.004*gully + 0.003*town + 0.003*competition + 0.003*family + 0.003*babae + 0.003*category + 0.003*offensive + 0.003*pc

6: 0.005*couch + 0.004*gricey + 0.004*faction 

In [33]:
#Unseen query
document= "Name a Filipino actor, comedian, politician and former professional basketball player in the Philippine Basketball Association?"

In [34]:
from nltk.tokenize import word_tokenize
tokens = word_tokenize(document)
doc_hdp = hdpmodel[hdpmodel[id2word.doc2bow(tokens)]]

In [35]:
#Predicted document(answer)
print(doc_hdp)

[(94, 0.4711542772450653)]


In [36]:
#Predicted document(answer)
print(df.iloc[94]['abstract'])

Marina Augusta Pepper (née Baker; born 8 December 1967)  is an English Liberal Democrat local politician, journalist, children's book author and former model and actress.
